# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 27 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [ ]:
!pip install torchsummaryX wandb --quiet

     |████████████████████████████████| 2.0 MB 29.2 MB/s 
     |████████████████████████████████| 184 kB 104.7 MB/s 
     |████████████████████████████████| 181 kB 93.9 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 178 kB 103.4 MB/s 
     |████████████████████████████████| 177 kB 100.6 MB/s 
     |████████████████████████████████| 174 kB 106.0 MB/s 
     |████████████████████████████████| 173 kB 95.4 MB/s 
     |████████████████████████████████| 168 kB 110.4 MB/s 
     |████████████████████████████████| 168 kB 107.4 MB/s 
     |████████████████████████████████| 166 kB 97.8 MB/s 
     |████████████████████████████████| 166 kB 101.2 MB/s 
     |████████████████████████████████| 162 kB 101.3 MB/s 
     |████████████████████████████████| 162 kB 107.4 MB/s 
     |████████████████████████████████| 158 kB 107.4 MB/s 
     |████████████████████████████████| 157 kB 105.3 MB/s 
     |████████████████████████████████| 157 kB 108.1 MB/s 
    

In [ ]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
from torch.cuda.amp import GradScaler, autocast
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"nikhilcg2603","key":"68c8ee5d31a123a0b0857ccf6e7c5752"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 7.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73274 sha256=08147c2596a485bf2241bc1f760a0366f3a65d52f7bbbc1439eb2038e18b589f
  Stored in directory: /root/.cache/pip/wheels/f3/67/7b/a6d668747974998471d29b230e7221dd01330ac34faebe4af4
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
# commands to download data from kaggle

!kaggle competitions download -c 11-785-s23-hw1p2
!mkdir '/content/data'

!unzip -qo '11-785-s23-hw1p2.zip' -d '/content/data'

100% 16.0G/16.0G [00:59<00:00, 321MB/s]
100% 16.0G/16.0G [00:59<00:00, 287MB/s]


In [ ]:
!pip install speechpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import speechpy

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [ ]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, offset=0, partition= "dev-clean", full_data= True): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes
        self.offset     = offset
        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = (root+"/"+partition+"/mfcc")  
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = (root+"/"+partition+"/transcript") 

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = sorted(os.listdir(self.mfcc_dir))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    = sorted(os.listdir(self.transcript_dir)) 

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names) 

        self.mfccs, self.transcripts = [], []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(self.mfcc_dir+"/"+mfcc_names[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            
            mfcc = speechpy.processing.cmvn(mfcc)
        #   Load the corresponding transcript
            transcript  = np.load(self.transcript_dir+"/"+transcript_names[i]) # Remove [SOS] and [EOS] from the transcript 
            transcript = transcript[1:-1]
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)        
        
        if full_data == True:
          self.mfcc_dir1 = root+'/'+'dev-clean'+'/mfcc'
          self.transcript_dir1 = root+'/'+'dev-clean'+'/transcript'
          mfcc_names1 = os.listdir(self.mfcc_dir1)
          transcript_names1 = os.listdir(self.transcript_dir1)  

          for i in range(len(mfcc_names1)):
            mfcc = np.load(self.mfcc_dir1+"/"+mfcc_names1[i])
            mfcc = speechpy.processing.cmvn(mfcc)
            transcript  = np.load(self.transcript_dir1+"/"+transcript_names1[i]) # Remove [SOS] and [EOS] from the transcript 
            transcript = transcript[1:-1]

            self.mfccs.append(mfcc)
            self.transcripts.append(transcript) 

          ####

          self.mfcc_dir1 = root+'/'+'train-clean-360'+'/mfcc'
          self.transcript_dir1 = root+'/'+'train-clean-360'+'/transcript'
          mfcc_names1 = os.listdir(self.mfcc_dir1)
          transcript_names1 = os.listdir(self.transcript_dir1)  

          for i in range(len(mfcc_names1)):
            mfcc = np.load(self.mfcc_dir1+"/"+mfcc_names1[i])
            mfcc = speechpy.processing.cmvn(mfcc)
            transcript  = np.load(self.transcript_dir1+"/"+transcript_names1[i]) # Remove [SOS] and [EOS] from the transcript 
            transcript = transcript[1:-1]

            self.mfccs.append(mfcc)
            self.transcripts.append(transcript) 



        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2) x 27, (T2+2) x 27 before removing [SOS] and [EOS]
        
        # TODO: Concatenate all mfccs in self.mfccs such that 
        # the final shape is T x 27 (Where T = T1 + T2 + ...) 
       
        self.mfccs = np.concatenate(self.mfccs, axis=0)

        # TODO: Concatenate all transcripts in self.transcripts such that 
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts    = np.concatenate(self.transcripts, axis=0, dtype = 'object')
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.transcripts)

        # Take some time to think about what we have done. 
        # self.mfcc is an array of the format (Frames x Features). 
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. 
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), 'constant', constant_values=0) # TODO

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such. 
        # Hence, we map these phonemes to integers
      
        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        for i in range(len(self.transcripts)):
            self.transcripts[i] = self.phonemes.index(self.transcripts[i]) 
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
       
        #start_i = ind + self.offset - self.context
        start_i = ind
        #end_i = ind + self.offset + self.context + 1
        end_i = ind + (2 * self.context) + 1
        frames = self.mfccs[start_i:end_i]
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        
        frames =  frames.flatten()# TODO: Flatten to get 1d data
        frames      = torch.FloatTensor(frames) # Convert to tensors
        
        phonemes    = torch.tensor(self.transcripts[ind])       

        return frames, phonemes

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self, root, phonemes = PHONEMES, context=0, offset=0, partition= "dev-clean"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes
        self.offset     = offset
        
        self.mfcc_dir       = (root+"/"+partition+"/mfcc")  
        mfcc_names          = sorted(os.listdir(self.mfcc_dir))
        self.mfccs = []

        for i in range(len(mfcc_names)):
            mfcc        = np.load(self.mfcc_dir+"/"+mfcc_names[i])
           

            mfcc = speechpy.processing.cmvn(mfcc)
            
            self.mfccs.append(mfcc)

        self.mfccs          = np.concatenate(self.mfccs, axis= 0)
        self.length = len(self.mfccs) #This is to account the padding that we did for mfcc. This 

        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), 'constant', constant_values=0)
    
    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        #start_i = ind + self.offset - self.context
        start_i = ind
        #end_i = ind + self.offset + self.context + 1
        end_i = ind + (2 * self.context) + 1
        frames = self.mfccs[start_i:end_i]
        
        
        frames =  frames.flatten()
        frames      = torch.FloatTensor(frames)
        return frames
    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

In [ ]:
config = {
    'epochs'        : 5,
    'batch_size'    : 2048,
    'context'       : 30,
    'init_lr'       : 1e-3,
    'architecture'  : 'double cylinder',
    'dropout'       : 0.15,
    'step_size'     : 1,
    'gamma'         : 0.05,
    'weight_decay'  : 1e-2,
    'T_max'         : 10,
    'eta_min'       : 0,
    'factor'        : 0.1 
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [ ]:
root = "/content/data/11-785-s23-hw1p2"
#TODO: Create a dataset object using the AudioDataset class for the training data 
train_data = AudioDataset(root,PHONEMES,context= config['context'],offset=0, partition="train-clean-100", full_data= True) 

# TODO: Create a dataset object using the AudioDataset class for the validation data 
val_data = AudioDataset(root,PHONEMES,context= config['context'], offset=0, partition="dev-clean", full_data= False) 

# TODO: Create a dataset object using the AudioTestDataset class for the test data 
test_data = AudioTestDataset(root,PHONEMES,context= config['context'],offset=0, partition= "test-clean") 

In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 4, #4
    batch_size  = config['batch_size'], 
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data, 
    num_workers = 2, #2
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data, 
    num_workers = 2, #2
    batch_size  = config['batch_size'], 
    pin_memory  = True, 
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*27)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  2048
Context        :  45
Input size     :  2457
Output symbols :  42
Train dataset samples = 168473356, batches = 82263
Validation dataset samples = 1928204, batches = 942
Test dataset samples = 1934138, batches = 945


In [ ]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([2048, 2457]) torch.Size([2048])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [ ]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(Network, self).__init__()
        # # self.model.name = "Cylinder 2048"
        # self.model = torch.nn.Sequential(
        #     torch.nn.Linear(input_size, 2048),
        #     torch.nn.BatchNorm1d(2048),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(2048,2048),
        #     torch.nn.BatchNorm1d(2048),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(2048,2048),
        #     torch.nn.BatchNorm1d(2048),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(2048,2048),
        #     torch.nn.BatchNorm1d(2048),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(2048,2048),
        #     torch.nn.BatchNorm1d(2048),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(2048,40),
        #     torch.nn.BatchNorm1d(40),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(40, output_size) #Change to 40,out if using the previous layer
        # )      

        # # self.model.name = "Cylinder 1024"
        # self.model = torch.nn.Sequential(
        #     torch.nn.Linear(input_size, 1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,1024),
        #     torch.nn.GELU(),
        #     # torch.nn.Dropout(p = config['dropout']),
        #     # torch.nn.Linear(2048,40),
        #     # torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024, output_size) #Change to 40,out if using the previous layer
        # )      

        # # self.model.name = "ZigZag 2048"
        # self.model = torch.nn.Sequential(
        #     torch.nn.Linear(input_size, 2048),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(2048,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,2048),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(2048,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,2048),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(2048,1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,2048),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(2048,1024),
        #     torch.nn.GELU(),
        #     # torch.nn.Dropout(p = config['dropout']),
        #     # torch.nn.Linear(2048,40),
        #     # torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024, output_size) #Change to 40,out if using the previous layer
        # )

         # self.model.name = "zipper 2000"
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),
            torch.nn.Linear(1450,2000),
            torch.nn.BatchNorm1d(2000),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),
            torch.nn.Linear(2000,1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),
            torch.nn.Linear(1450,2000),
            torch.nn.BatchNorm1d(2000),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),
            torch.nn.Linear(2000,1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),
            torch.nn.Linear(1450,2000),
            torch.nn.BatchNorm1d(2000),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),
            torch.nn.Linear(2000,1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),
            torch.nn.Linear(1450,50),
            torch.nn.BatchNorm1d(50),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),
            torch.nn.Linear(50, output_size) #Change to 40,out if using the previous layer
        )      

        #  # self.model.name = "hour glass 4096"
        # self.model = torch.nn.Sequential(
        #     torch.nn.Linear(input_size, 2048),
        #     torch.nn.BatchNorm1d(2048),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(2048,1024),
        #     torch.nn.BatchNorm1d(1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,512),
        #     torch.nn.BatchNorm1d(512),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(512,256),
        #     torch.nn.BatchNorm1d(256),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(256,512),
        #     torch.nn.BatchNorm1d(512),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(512,1024),
        #     torch.nn.BatchNorm1d(1024),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1024,2048),
        #     torch.nn.BatchNorm1d(2048),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(2048, output_size) #Change to 40,out if using the previous layer
        # )    

        # # self.model.name = "Double Cylinder 1500"
        # self.model = torch.nn.Sequential(
        #     torch.nn.Linear(input_size, 1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     # torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     # torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     # torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,750),
        #     torch.nn.BatchNorm1d(750),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(750,750),
        #     torch.nn.BatchNorm1d(750),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(750,750),
        #     torch.nn.BatchNorm1d(750),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(750,750),
        #     torch.nn.BatchNorm1d(750),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(750,750),
        #     torch.nn.BatchNorm1d(750),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(750,750),
        #     torch.nn.BatchNorm1d(750),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(750,750),
        #     torch.nn.BatchNorm1d(750),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(750,656),
        #     torch.nn.BatchNorm1d(656),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(656, output_size) #Change to 40,out if using the previous layer
        # )        

        # # self.model.name = "Cylinder 1500"
        # self.model = torch.nn.Sequential(
        #     torch.nn.Linear(input_size, 1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500,1350),
        #     torch.nn.BatchNorm1d(1350),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1350, output_size) #Change to 40,out if using the previous layer
        # )      

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

In [ ]:
def initialize(model):
  if type(model)==torch.nn.Linear:
    torch.nn.init.xavier_normal(model.weight,gain=1.0)
    # torch.nn.init.kaiming_normal_(model.weight, mode= 'fan_in')

Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [ ]:
INPUT_SIZE  = (2*config['context'] + 1) * 27 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
model.apply(initialize)
summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

                         Kernel Shape  Output Shape    Params Mult-Adds
Layer                                                                  
0_model.Linear_0         [2457, 1500]  [2048, 1500]    3.687M   3.6855M
1_model.BatchNorm1d_1          [1500]  [2048, 1500]      3.0k      1.5k
2_model.GELU_2                      -  [2048, 1500]         -         -
3_model.Dropout_3                   -  [2048, 1500]         -         -
4_model.Linear_4         [1500, 1500]  [2048, 1500]   2.2515M     2.25M
5_model.BatchNorm1d_5          [1500]  [2048, 1500]      3.0k      1.5k
6_model.GELU_6                      -  [2048, 1500]         -         -
7_model.Linear_7         [1500, 1500]  [2048, 1500]   2.2515M     2.25M
8_model.BatchNorm1d_8          [1500]  [2048, 1500]      3.0k      1.5k
9_model.GELU_9                      -  [2048, 1500]         -         -
10_model.Dropout_10                 -  [2048, 1500]         -         -
11_model.Linear_11       [1500, 1500]  [2048, 1500]   2.2515M   

/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[2457, 1500]","[2048, 1500]",3687000.0,3685500.0
1_model.BatchNorm1d_1,[1500],"[2048, 1500]",3000.0,1500.0
2_model.GELU_2,-,"[2048, 1500]",NaN,NaN
3_model.Dropout_3,-,"[2048, 1500]",NaN,NaN
4_model.Linear_4,"[1500, 1500]","[2048, 1500]",2251500.0,2250000.0
5_model.BatchNorm1d_5,[1500],"[2048, 1500]",3000.0,1500.0
6_model.GELU_6,-,"[2048, 1500]",NaN,NaN
7_model.Linear_7,"[1500, 1500]","[2048, 1500]",2251500.0,2250000.0
8_model.BatchNorm1d_8,[1500],"[2048, 1500]",3000.0,1500.0


In [ ]:
from contextlib import ExitStack
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function. 
# We use CE because the task is multi-class classification 

optimizer = torch.optim.AdamW(model.parameters(), lr= config['init_lr'], weight_decay= config['weight_decay']) #Defining Optimizer
# Recommended : Define Scheduler for Learning Rate, 

# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= config['step_size'], gamma= config['gamma']) #Defining StepLR Scheduler
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max= config['T_max'], eta_min= config['eta_min'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= 'min', factor= config['factor'], patience= 5, threshold= 1e-4)
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high? 
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html
scaler = torch.cuda.amp.GradScaler()

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

914

In [ ]:
from torch._C import dtype
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):
        
        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # with torch.cuda.amp.autocast('cuda'):
          ### Forward Propagation
        logits  = model(frames)

          ### Loss Calculation
        loss    = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward() 
        # scaler.scale(loss).backward()
        # scaler.step(optimizer)
        # scaler.update()

        ### Gradient Descent
        optimizer.step()       
        

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))), 
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

        # ### Mixed Precision

        # with autocast(device_type='cuda', dtype= torch.float16):
        #   output = model(input)
        #   loss = loss_fn(output, target)
        
        # scaler.scale(loss).backward()
        # scaler.step()
        # scaler.update()
  
    
    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)
    scheduler.step(tloss) #adding scheduler plateau function

    return tloss, tacc

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode(): 
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]
        
        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))), 
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()
    
        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="f303efa63722be981833b9f7e8dd7bb0efd47e1f") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name    = "nchinnal-run25-460", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    # reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    id     = "jfc42chs", ### Insert specific run id here if you want to resume a previous run
    resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account 
    config  = config ### Wandb Config for your run
)

In [ ]:
### Save your model architecture as a string with str(model) 
model_arch  = str(model)

### Save it in a txt file 
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20230217_034508-jfc42chs/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run 
    # Optionally, you can log at each batch inside train/eval functions 
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss, 
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
# if best_acc < val_acc:
    # best_acc = val_acc
save_path = "best_model_{0}_{1}_{2}.pth".format(config['init_lr'],config['batch_size'],config['context'])
torch.save({'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict()},
            '/content/'+save_path)
### Finish your wandb run
run.finish()


Epoch 1/5


Train:   0%|          | 0/82263 [00:00<?, ?it/s]

Val:   0%|          | 0/942 [00:00<?, ?it/s]

	Train Acc 87.6180%	Train Loss 0.3556	 Learning Rate 0.0010000
	Val Acc 89.2162%	Val Loss 0.2988

Epoch 2/5


Train:   0%|          | 0/82263 [00:00<?, ?it/s]

...

	Train Acc 87.7595%	Train Loss 0.3508	 Learning Rate 0.0010000
	Val Acc 89.4303%	Val Loss 0.2936

Epoch 5/5


Train:   0%|          | 0/82263 [00:00<?, ?it/s]

Val:   0%|          | 0/942 [00:00<?, ?it/s]

	Train Acc 87.7950%	Train Loss 0.3496	 Learning Rate 0.0010000
	Val Acc 89.4181%	Val Loss 0.2931


lr,▁▁▁▁▁
train_acc,▁▃▅▇█
train_loss,█▆▄▂▁
val_acc,▁▆▃██
valid_loss,█▄▇▂▁
lr,0.001
train_acc,87.79504
train_loss,0.34957
val_acc,89.41815
valid_loss,0.29306


# Testing and submission to Kaggle

---



Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
best_acc = val_acc

In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)             
            
            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits, dim=1)

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
            # TODO
            test_predictions.extend(predicted_phonemes.tolist())

    return test_predictions
    

In [ ]:
#Load Best Model ##
check_point = torch.load('/content/'+save_path)
model.load_state_dict(check_point['model_state_dict'])

<All keys matched successfully>

In [ ]:
predictions = test(model, test_loader)

  0%|          | 0/945 [00:00<?, ?it/s]

In [ ]:
predictions = [PHONEMES[x] for x in predictions]

In [ ]:
len(predictions)

1934138

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11-785-s23-hw1p2 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle

100% 19.3M/19.3M [00:00<00:00, 30.8MB/s]
Successfully submitted to Frame-Level Speech Recognition